In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow_addons as tfa

# from tqdm import tqdm
# from random import choices

In [ ]:
# file path for saved model
file_saved_model = '../input/simplenn/es_model_final_4_4096_0.25_1024.h5'
file_saved_fmeans = '../input/fmeans/fmeans.csv' 

In [ ]:
# train a new model or load a saved model
training = True
param_tuning = False
acc = 'tpu'

# init TPU/GPU strategy
if(training):
    if(acc == 'tpu'):
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        # instantiate a distribution strategy
        acc_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        acc_strategy = tf.distribute.MirroredStrategy()
    print(acc_strategy.num_replicas_in_sync)

In [ ]:
%%time
# Load data, somehow tpu complains about datatable
if acc == 'tpu':
    train_data = pd.read_csv('../input/jane-street-market-prediction/train.csv')
else:
    import datatable as dt
    train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
    train_data = train_data_datatable.to_pandas()
    del train_data_datatable
    gc.collect()

In [ ]:
label = 'action'
features = [c for c in train_data.columns if 'feature' in c]
train_data['action'] = ((train_data['resp'].values) > 0).astype('int') 
train_data.shape

In [ ]:
train_data['action_3'] = ((train_data['resp_3'].values) > 0).astype('int') 
train_data['action_4'] = ((train_data['resp_4'].values) > 0).astype('int') 
train_data['action_1'] = ((train_data['resp_1'].values) > 0).astype('int') 
train_data['action_2'] = ((train_data['resp_2'].values) > 0).astype('int') 

In [ ]:
%%time
if(training):
    train_data = train_data[train_data['weight'] > 0]
    # fill nan with feature mean
    feature_means = train_data[features].mean()
    train_data.fillna(feature_means,inplace=True)
    feature_means.to_csv('fmeans.csv') # save means for use in submission
    # first 85 days excluded
    train_data = train_data[train_data['date']>85]
    train_data.reset_index(drop=True,inplace=True)

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
            

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
from matplotlib.colors import ListedColormap

def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
if(True):
    cv = GroupTimeSeriesSplit()
    X = train_data[features]
    y = train_data[label]
    y_resp = train_data[['action','action_1','action_2','action_3','action_4']]
    groups = train_data['date'].values
    weights = train_data['weight'].values
    resp = train_data['resp'].values

In [ ]:
del train_data
gc.collect()

In [ ]:
# plot CV data split
fig, ax = plt.subplots()

plot_cv_indices(
    cv,
    X,
    y,
    groups,
#     X=train_data.loc[train_data['date'] <100,features].values,
#     y=train_data.loc[train_data['date'] <100,label].values,
#     group =train_data.loc[train_data['date']<100,'date'].values,
    ax=ax,
    n_splits = 5,
    lw=20
)

In [ ]:
# method to set up the CNN architecture
def create_nn(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)

    return model

In [ ]:
# Define cv_scoring function for param tuning
def cv_scoring(params):
    # params
    depth = params['depth']
    units_per_layer = params['units_per_layer']
    dropout_rate = params['dropout_rate']
    learning_rate = params['learning_rate']
    BATCH_SIZE = int(params['batch_size']) #1024 #4096 #128 * tpu_strategy.num_replicas_in_sync #4096
    EPOCHS = int(params['epochs']) #200
    label_smoothing = params['label_smoothing']   
    
    hidden_units = np.ones(int(depth)) * int(units_per_layer)
    dropout_rates = np.ones(int(depth+1)) * dropout_rate
    
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    histories = []
    models = []
    
    print('Starting cross-val ...')
    # cross-val with group time split
    for i, (train_idx, valid_idx) in enumerate(cv.split(
                                                X = X.values,
                                                y = y.values,
                                                groups=groups)):
        print('Fold number ' +  str(i+1) +'...')
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[valid_idx]
        y_val = y.iloc[valid_idx]
            
        with acc_strategy.scope():
            my_model = create_nn(num_columns = len(features),
                                num_labels = 1,
                                hidden_units = hidden_units,
                                dropout_rates = dropout_rates,
                                label_smoothing = label_smoothing,
                                learning_rate = learning_rate)    
            # compile model
            my_model.compile(
                optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
                metrics=tf.keras.metrics.AUC(name="AUC"),
                )
        # fit model        
        history = my_model.fit(X_train, y_train,
                               epochs = EPOCHS,
                               batch_size = BATCH_SIZE,
                               validation_data = (X_val, y_val),
                               callback = [early_stopping], 
                               verbose = 1
                              )
        histories.append(history)  
        models.append(my_model)
        print('Saved models and history for fold ' + str(i+1))
    
    print('Finished cross-val.')
    
    final_val_accs = [history.history['val_AUC'][-1] for histroy in histories]       
    print(np.average(final_val_accs))
    
    return np.average(final_val_accs) * -1.0 #, histories, models



In [ ]:
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope

param_space = {
    'depth' : hp.quniform('depth', 2, 3, 1),
    'units_per_layer' : hp.choice("units_per_layer", [128, 256]),
    'dropout_rate' : hp.choice("dropout_rate", [0.25,0.5, 0.75]),
    'label_smoothing' : 1e-2,
    'learning_rate' : hp.loguniform('learning_rate', 0.001, 0.1),
    'batch_size' : hp.choice('batch_size', [8, 16, 32, 64, 124]), #1024 #4096 #128 * tpu_strategy.num_replicas_in_sync #4096
    'epochs' : 10 #200
}

In [ ]:
%%time
# Run optimiser
if(param_tuning):
    
    max_evals = 50
    trials = Trials()
    hopt = fmin(fn = cv_scoring,
                space = param_space, 
                algo = tpe.suggest, 
                max_evals = max_evals, ## 50
                trials = trials, 
               )

In [ ]:
if(param_tuning):
    params_to_plot = ['depth', 'units_per_layer', 'learning_rate', 'dropout_rate', 'batch_size']
    fig, ax = plt.subplots(len(params_to_plot),1, figsize= (25,20))
    fig.subplots_adjust(hspace=0.5)
    for i,param in enumerate(params_to_plot):
        xs = [t['tid'] for t in trials.trials]
        ys = [t['misc']['vals'][param] for t in trials.trials]
        ax[i].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
        ax[i].set_title(str(param) + ' vs t ', fontsize=18)
        ax[i].set_xlabel('id', fontsize=16)
        ax[i].set_ylabel(str(param), fontsize=16)

In [ ]:
if(param_tuning):
    f, ax = plt.subplots(1)
    xs = [t['tid'] for t in trials.trials]
    ys = [t['result']['loss'] for t in trials.trials]
    ax.scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
    ax.set_title('loss over time', fontsize=18)
    ax.set_xlabel('trials', fontsize=16)
    ax.set_ylabel('loss', fontsize=16)

In [ ]:
%%time
# Simple cross-val training rounds

from keras.callbacks import ReduceLROnPlateau

if(training):
    # params
    depth = 4
    units_per_layer = int(1024) #128 #256 #512
    dropout_rate = 0.15
    label_smoothing = 1e-2
    learning_rate = 1e-3
    BATCH_SIZE = 4096 #4096 #1024 #4096 #128 * tpu_strategy.num_replicas_in_sync #4096
    EPOCHS = 75 #75
    #STEPS_PER_EPOCH = X.shape[0] // BATCH_SIZE
    
    hidden_units = np.ones(depth) * units_per_layer
#     hidden_units = [124,124,124] # [2, 2, 2]
    dropout_rates = np.ones(depth+1) * dropout_rate
#     dropout_rates = [0.5, 0.5, 0.5, 0.5]
#     hidden_units = [384, 896, 896, 394]
#     dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
    histories = []
    models = []
    
    print('Starting cross-val ...')
    # cross-val with group time split
    for i, (train_idx, valid_idx) in enumerate(cv.split(
                                                X = X.values,
                                                y = y_resp.values,
                                                groups=groups)):
        print('Fold number ' +  str(i+1) +'...')
        X_train = X.iloc[train_idx]
        y_train = y_resp.iloc[train_idx]
        X_val = X.iloc[valid_idx]
        y_val = y_resp.iloc[valid_idx]
        
        rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, min_delta = 1e-4, mode = 'max')
        es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 10, mode = 'max', baseline = None, restore_best_weights = True, verbose=0)
        ckp = ModelCheckpoint(f'es_model_fold_{i}.h5', monitor='val_AUC', verbose=0, save_best_only = True, save_weights_only = True, mode = 'max')

        with acc_strategy.scope():
            my_model = create_nn(num_columns = len(features),
                                num_labels = 5,
                                hidden_units = hidden_units,
                                dropout_rates = dropout_rates,
                                label_smoothing = label_smoothing,
                                learning_rate = learning_rate)    
            # compile model
            my_model.compile(
                optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
                metrics=tf.keras.metrics.AUC(name="AUC"),
                )

        # fit model        
        history = my_model.fit(X_train, y_train,
                               epochs = EPOCHS,
                               batch_size = BATCH_SIZE,
                               validation_data = (X_val, y_val),
                               verbose = 1,
                               callbacks = [es,ckp]
                              )
        histories.append(history)  
#         models.append(my_model)
        print('Saved models and history for fold ' + str(i+1))
    
    print('Finished cross-val.')

In [ ]:
# Plot training and validation accuracy curves
if(training):
    counter = 0
    for history in histories:
        counter += 1
        train_loss = history.history['loss']
        val_loss = history.history['val_loss']
        train_acc = history.history['AUC']
        val_acc = history.history['val_AUC']

        epochs = range(0,len(train_loss))

        fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,2))
        fig.suptitle('Training progress in fold ' + str(counter))
        fig.subplots_adjust(wspace=0.3)

        ax1.plot(epochs,train_loss,label = 'train')
        ax1.plot(epochs,val_loss,label = 'validation')
        ax1.set_ylabel('Loss')
        ax2.plot(epochs,train_acc,label = 'train')
        ax2.plot(epochs,val_acc,label = 'validation')
        ax2.set_ylabel('AUC')
        ax1.legend()
        ax2.legend()

    plt.show()
    val_accs = [history.history['val_AUC'][-1] for history in histories]
    val_best = [max(history.history['val_AUC']) for history in histories]
    print('Average last AUC ' + str(np.average(val_accs)))
    print('StDev last AUC '  + str(np.std(val_accs)))
    print('AUC last fold '  + str(val_accs[-1]))
    print('Average max AUC ' + str(np.average(val_best)))
    print('StDev max AUC '  + str(np.std(val_best)))
    print('max AUC last fold '  + str(val_best[-1]))

In [ ]:
%%time
# Train final model

X_train, X_val, y_train, y_val = train_test_split(X,y_resp,test_size = 0.2,shuffle=False)

if(training):
    # params for final model
    hidden_units = [1024,1024,1024, 1024] # [2, 2, 2]
    dropout_rates = [0.25, 0.25, 0.25, 0.25, 0.25]
    label_smoothing = 1e-2
    learning_rate = 1e-3
    
    rlr = ReduceLROnPlateau(monitor = 'val_AUC', factor = 0.1, patience = 3, verbose = 0, min_delta = 1e-4, mode = 'max')
    es = EarlyStopping(monitor = 'val_AUC', min_delta = 1e-4, patience = 10, mode = 'max', baseline = None, restore_best_weights = True, verbose=0)
    ckp = ModelCheckpoint(f'es_model_final.h5', monitor='val_AUC', verbose=0, save_best_only = True, save_weights_only = False, mode = 'max')


    with acc_strategy.scope():
        final_model = create_nn(num_columns = len(features),
                            num_labels = 5,
                            hidden_units = hidden_units,
                            dropout_rates = dropout_rates,
                            label_smoothing = label_smoothing,
                            learning_rate = learning_rate)    
        # compile model
        final_model.compile(
                optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
                metrics=tf.keras.metrics.AUC(name="AUC"),
                )
    
    BATCH_SIZE = 4096 #128 * tpu_strategy.num_replicas_in_sync #4096
    EPOCHS = 75 

    final_history = final_model.fit(X_train,y_train,epochs = EPOCHS, batch_size = BATCH_SIZE,
                               validation_data = (X_val, y_val),
                               verbose = 1,
                               callbacks = [rlr, es,ckp])

In [ ]:
# Plot training and validation accuracy curves
if(training):
    train_loss = final_history.history['loss']
    val_loss = final_history.history['val_loss']
    train_acc = final_history.history['AUC']
    val_acc = final_history.history['val_AUC']

    epochs = range(0,len(train_loss))

    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,2))
    fig.suptitle('Training progress')
    fig.subplots_adjust(wspace=0.3)

    ax1.plot(epochs,train_loss,label = 'train')
    ax1.plot(epochs,val_loss,label = 'validation')
    ax1.set_ylabel('Loss')
    ax2.plot(epochs,train_acc,label = 'train')
    ax2.plot(epochs,val_acc,label = 'validation')
    ax2.set_ylabel('AUC')
    ax1.legend()
    ax2.legend()

    plt.show()
    
    val_auc = final_history.history['val_AUC']
    print('AUC last fold '  + str(val_auc[-1]))
    print('best AUC '  + str(max(val_auc)))

In [ ]:
# Save model
if(training):
    final_model.save(f'model_final.h5')

In [ ]:
del X, y
gc.collect()

In [ ]:
# load model    
if(training):
    saved_model = final_model
else:
    saved_model = keras.models.load_model(file_saved_model)
    feature_means = pd.read_csv(file_saved_fmeans, index_col = 0)

In [ ]:
saved_model.summary()

In [ ]:
%%time
y_val_probs = np.array(saved_model(X_val[features].values, training = False))

In [ ]:
prob_r = np.array([prob[0] for prob in y_val_probs])
prob_r3 =  np.array([prob[1] for prob in y_val_probs])
prob_r4 =  np.array([prob[2] for prob in y_val_probs])
pred_r = np.array(np.where(prob_r >= 0.5,1,0))
pred_r3 = np.array(np.where(prob_r3 >= 0.5,1,0))
pred_r4 = np.array(np.where(prob_r4 >= 0.5,1,0))

In [ ]:
pred_r = np.array(np.where(prob_r >= 0.5,1,0))
roc_auc_score(y_val['action'],pred_r)

In [ ]:
preds_df = pd.DataFrame(np.transpose(np.array([pred_r,pred_r3,pred_r4])))

In [ ]:
preds_df.corr()

In [ ]:
y_val.corr()

In [ ]:
# # code up the evaluation function
# def evaluate_util(X):
#     grouped = X.groupby('date')
#     ## depends on definition of action ##
#     pi = grouped.apply(lambda x : np.sum(x['weight'] * x['resp'] * x['action_pred']) )
#     t = np.sum(pi)/np.sqrt(np.sum(np.power(pi,2))) * np.sqrt(250)/len(pi)
#     u = np.minimum(np.maximum(t,0),6) * np.sum(pi)
#     return u

In [ ]:
# X['action_pred'] = y
# X['date'] = groups
# X['weight'] = weights
# X['resp'] = resp
# evaluate_util(X)

In [ ]:
# %%time
# # X_train, X_val, y_train, y_val = train_test_split(X,y_resp,test_size = 0.2,shuffle=False)
# # y_val_probs = saved_model.predict(X[features]) 
# y_val_probs = saved_model(X[features], training = False) 
# y_probs_resp = [ i for i,j,k in y_val_probs]

In [ ]:
# %%time
# # check if 0.5 is optimal threshold
# # baseline eval score
# X['action_pred'] = y
# print('Base line' + str(evaluate_util(X)))
# thresholds = [0.475 + i/1000 for i in range(0,50)]
# eval_scores = {}
# for th in thresholds:
#     X_val['action_pred'] = np.where(np.array(y_probs_resp) >= th, 1, 0).astype(int)
#     eval_scores[th] = evaluate_util(X_val)
# th_opt = max(eval_scores, key=eval_scores.get)
# print(th_opt)
# print(max(eval_scores.values()))
# scores = [x for x in eval_scores.values()]
# plt.show()

In [ ]:
# check if it's working
if(training):
#     saved_model = saved_models[0]
    example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
    example_test[features] = example_test[features].fillna(method = 'ffill').fillna(0)
    saved_model(example_test[features].values,training=False)

In [ ]:
# prepare fill function
from numba import njit
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
# Jane Street submission environment
if(not training):
    import janestreet
    env = janestreet.make_env() # initialize the environment
    iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
# sample_prediction_df.action = 0
# env.predict(sample_prediction_df)

In [ ]:
%%time
th_opt = 0.501
if(not training):
    for (test_df, sample_prediction_df) in iter_test:   
        if test_df['weight'].values[0] > 0:
            x_tst = test_df.loc[:, features].values
            x_tst[0,:] = fast_fillna(x_tst[0,:], feature_means.values.ravel())
            pred = saved_model(x_tst,training=False)
#             pred = 0
#             for model in saved_models:
#                 pred = pred + model(x_tst, training = False)/len(saved_models)
            sample_prediction_df.action = np.where(pred >= th_opt, 1, 0).astype(int)
        else:
            sample_prediction_df.action = 0
        env.predict(sample_prediction_df)